In [3]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
df_prc = pd.read_csv("./close.csv", index_col=0, parse_dates=True)
df_logret = df_prc.apply(np.log).diff().iloc[1:]

In [ ]:
x_logret = torch.Tensor(df_logret.values)

In [ ]:
lookback = 5
ts = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=lookback, padding_mode='zeros', padding=lookback-1, bias=False)
ts.weight.data = ts.weight*0 + 1
# ts_operator
x_logret_ts = ts(x_logret.T[:,torch.newaxis,:]).transpose(0,1)[0,:,:-lookback+1].T#.shape

In [ ]:
x = x_logret
cs = nn.Linear(in_features=x.shape[1], out_features=x.shape[1], bias=False)
cs.weight.data = cs.weight*0 + 1
# cs operator
x_logret_cs = cs(x_logret)

In [ ]:
# without batch
lookback = 5
x = x_logret

ts_layer = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=lookback, padding_mode='zeros', padding=lookback-1, bias=False)
cs_layer = nn.Linear(in_features=x.shape[1], out_features=x.shape[1], bias=False)
ts = lambda x: ts_layer(x.T[:,torch.newaxis,:]).transpose(0,1)[0,:,:-lookback+1].T
cs = lambda x: cs_layer(x)
csts_output = ts(cs(x))
csts_output = (csts_output - csts_output.mean(axis=1, keepdim=True))
csts_output = csts_output / csts_output.abs().sum(axis=1, keepdim=True) # long 0.5 short 0.5 
alpha = csts_output

In [ ]:
# batch size = n_alpha
lookback = 5
n_alpha = 30
x = x_logret

ts_layer = nn.Conv1d(in_channels=n_alpha, out_channels=n_alpha, groups=n_alpha, kernel_size=lookback, padding_mode='zeros', padding=lookback-1, bias=False)
cs_layer = nn.Linear(in_features=x.shape[1], out_features=x.shape[1], bias=False) # depthwise
ts = lambda x: ts_layer(x.permute(2,0,1)).permute(1,2,0)[:,:-lookback+1,:]
cs = lambda x: cs_layer(x)
x_repeat = x.repeat(n_alpha,1,1) # batch, time, instrument
csts_output = ts(cs(x_repeat))  
csts_output = (csts_output - csts_output.mean(axis=2, keepdim=True))
csts_output = csts_output / csts_output.abs().sum(axis=2, keepdim=True) # long 0.5 short 0.5 
alpha_returns = csts_output * x_repeat
alpha_returns = alpha_returns.sum(axis=2)

In [ ]:
alpha_cov = alpha_returns.cov()
alpha_eigval, alpha_eigvec = torch.linalg.eigh(alpha_cov)
alpha_weights = sum([a*v for a,v in zip(torch.sqrt(alpha_eigval), alpha_eigvec.T)])
alpha_weights = alpha_weights / (alpha_weights.abs().sum())

In [ ]:
raw_weights = (csts_output * (alpha_weights[:,torch.newaxis,torch.newaxis])).sum(axis=0)
returns = (alpha_returns * alpha_weights[:,np.newaxis]).sum(axis=0)
# returns_val = (raw_weights * x_logret).sum(axis=1)
# assert ( returns_val - returns ).abs().max() < 1e-6

In [208]:
shp = returns.mean() / returns.std()

In [75]:
ret = alpha * x
ret.sum(axis=1)

tensor([-0.0032, -0.0008, -0.0003,  ...,  0.0035, -0.0021, -0.0019],
       grad_fn=<SumBackward1>)

In [ ]:
eval, evec = torch.linalg.eigh(x_tf.T.cov()) # eval[i] , evec[:,i]
# x_tf.T.cov() - sum([a*(v[:,torch.newaxis]@v[torch.newaxis,:]) for a,v in zip(eval,evec.T)])

In [ ]:
n_factor = 30
weights = sum([v/torch.sqrt(a) for a,v in zip(eval[-n_factor:],evec.T[-n_factor:])])
normalize = weights.abs().sum() * 0.5 # long 1 short 1
weights = weights / normalize 
variance = n_factor / normalize # weights @ (x_tf.T.cov()) @ weights


In [ ]:
x@ weights

tensor([-2.2009e-04,  3.8787e-04,  1.3604e-03, -1.4446e-02,  3.0492e-03,
         6.7464e-04,  7.7460e-03,  2.1189e-03, -9.9293e-03,  1.6725e-03,
        -9.1427e-03, -1.7301e-02, -7.5083e-03, -5.1239e-03,  1.3681e-02,
        -1.7822e-02,  2.2641e-03,  1.1435e-02, -9.4208e-03, -1.3212e-02,
        -1.3921e-02,  9.1543e-03, -9.4438e-03,  1.2114e-02,  2.2472e-02,
         7.3175e-03,  3.5766e-03,  1.0507e-02,  8.2215e-03,  1.1521e-03,
        -1.3140e-02,  2.6374e-03,  8.3662e-03, -1.1318e-02, -3.7795e-03,
        -5.0300e-03, -5.4616e-04, -1.3189e-02,  1.4785e-02, -4.6221e-03,
        -2.1622e-02,  1.7472e-02, -2.0424e-02, -3.5132e-02,  1.3000e-03,
         6.2976e-03, -1.9479e-02, -5.5162e-03, -5.8511e-03,  1.6059e-02,
        -1.2325e-02,  7.7561e-04,  1.1482e-02, -9.1925e-03, -4.3449e-03,
         8.8252e-03,  1.9328e-02, -1.3388e-02,  1.0383e-02,  1.7444e-02,
         2.4698e-03, -8.1510e-04,  1.3505e-02,  1.6301e-02, -7.1803e-03,
         3.0053e-04, -1.8014e-02, -8.5565e-03,  3.3

In [ ]:

# torch.bilinear()

tensor(8.5498e-06, grad_fn=<DotBackward0>)

In [68]:

class MinVar(nn.Module):
    def __init__(self, cov, gross_exposure_constraint, use_latent=False, cuda=True):
        super().__init__()
        self.enable_cuda = cuda
        self.cov = cov.cuda() if cuda else cov
        num_instrument = cov.shape[0]
        self.use_latent = use_latent
        if use_latent:
            self.latent = nn.Linear(in_features=30, out_features=num_instrument, bias=False) # A
            self.fc = nn.Linear(in_features=30, out_features=1, bias=False) # w_e
        else:
            self.fc = nn.Linear(in_features=num_instrument, out_features=1, bias=False) # w_e
        self.fc.weight.data = (torch.rand_like(self.fc.weight) - 0.5) / num_instrument 
        self.gross_exp_constr = gross_exposure_constraint

        eigenvalues, eigenvectors = torch.linalg.eigh(self.cov)
        sorted_indices = torch.argsort(eigenvalues, descending=True)
        self.eigenvalues = eigenvalues[sorted_indices]
        self.eigenvectors = eigenvectors[:, sorted_indices]

    @property
    def portfolio_weight(self): # w_p = v * A * w_e
        if self.use_latent:
            weight_in_eigenvalue_space = self.latent(self.fc.weight).T
        weight_in_eigenvalue_space = self.fc.weight.T
        weight_in_instrument_space = self.eigenvectors @ weight_in_eigenvalue_space
        norm_coef = self.normalize_coeff(weight_in_instrument_space)
        return weight_in_eigenvalue_space / norm_coef, weight_in_instrument_space / norm_coef
    
    def get_weight(self):
        return self.portfolio_weight[1].cpu().detach().numpy()

    @staticmethod
    def normalize_coeff(x):
        # 1x Long
        return x.sum()
    
    def _normalize_weight(self):
        # Prevents divergence
        if self.use_latent:
            self.latent.weight.data = self.latent.weight / torch.linalg.matrix_norm(self.latent.weight, ord='fro')
        # self.fc.weight.data = self.fc.weight / torch.linalg.vector_norm(self.fc.weight, ord=2)
        self.fc.weight.data = self.fc.weight / self.normalize_coeff(self.fc.weight)

    def fit(self, n_epochs=10000, print_step=1000, learning_rate=1e-4):
        if self.enable_cuda:
            self.cuda()
        optimizer = torch.optim.Adagrad(self.parameters(), lr=learning_rate) # Momentum-less optimizer for barrier method
        for e in range(n_epochs):
            w_e, w_i = self.portfolio_weight
            gross_exposure = torch.linalg.vector_norm(w_i, ord=1)
            portfolio_variance = self.eigenvalues @ (w_e.square())
            barrier_loss = torch.relu(gross_exposure - self.gross_exp_constr)
            loss = torch.log(portfolio_variance) + barrier_loss

            loss.backward()
            optimizer.step()
            # self._normalize_weight()
            print(f'{str(e).zfill(5)}/{n_epochs}, variance={portfolio_variance.item():.2e}, gross_exposure = {gross_exposure.item():.2e}, barrier_loss={barrier_loss:.2e}, loss={loss.item():.2e}, normalization={torch.sum(self.fc.weight):.2e}     ', end='\r', flush=True)
            if print_step and e%print_step == 0 :
                print()
        print("\nTrain Finished")